In [1]:
import os
from os import path

import numpy as np
import torch
from torch import optim
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torchvision
from torchvision import transforms

from knockoff import datasets
import knockoff.config as cfg
from knockoff.adversary.policies import RandomAdversary
from knockoff.victim.blackbox import Blackbox
from knockoff.adversary.train import samples_to_transferset, get_optimizer
from knockoff.models import zoo
import knockoff.utils.model as model_utils

import time
import copy


In [5]:
device = torch.device("cuda")

np.random.seed(cfg.DEFAULT_SEED)
torch.manual_seed(cfg.DEFAULT_SEED)
torch.cuda.manual_seed(cfg.DEFAULT_SEED)

data_transform = transforms.Compose(
    [
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)
val_dataset = torchvision.datasets.ImageNet("/data/shjun/imagenet2012", split="val", transform=data_transform)
train_dataset = torchvision.datasets.ImageNet("/data/shjun/imagenet2012", split="train", transform=data_transform)
train_dataset_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)


TypeError: __init__() got an unexpected keyword argument 'device'

In [3]:
blackbox = torchvision.models.resnet34(pretrained=True).to(device).eval()
thief = torchvision.models.resnet101(pretrained=False).to(device).train()

In [11]:
for i, o in train_dataset_loader:
    inf = blackbox(i.to(device))
    print(o, torch.argmax(inf, dim=1))
    break

tensor([506, 654, 557, 775, 114, 743, 874, 858]) tensor([506, 785, 733, 775, 114, 743, 874, 858], device='cuda:0')
